# JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

****
### Dataset

+ A JSON file containing data about projects funded by the World Bank 

## Part 1 - Find the 10 countries with the most projects

In [1]:
# Import packages
import json
import pandas as pd
from collections import Counter

In [13]:
# load the JSON file as a Pandas dataframe
json_df = pd.read_json('data/world_bank_projects.json')

In [4]:
# Count the values in the countryname column
json_df.countryname.value_counts().head(10)

Republic of Indonesia              19
People's Republic of China         19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
Kingdom of Morocco                 12
People's Republic of Bangladesh    12
Republic of Mozambique             11
Africa                             11
Name: countryname, dtype: int64

As we can see, Indonesia and China tie for the most World Bank projects.

*****
## Part 2 - Top 10 Major Project Themes

In [5]:
# Show the column where the Major Project Themes are stored
json_df.mjtheme_namecode.head()

0    [{'code': '8', 'name': 'Human development'}, {...
1    [{'code': '1', 'name': 'Economic management'},...
2    [{'code': '5', 'name': 'Trade and integration'...
3    [{'code': '7', 'name': 'Social dev/gender/incl...
4    [{'code': '5', 'name': 'Trade and integration'...
Name: mjtheme_namecode, dtype: object

We see that the Major Project Themes are in a list of dictionaries in the column mjtheme_namecode.

To count the Major Project Themes, we must loop through each dictionary in each row.

In [15]:
# Create a counter for projects by each major theme
projects_by_mjtheme = Counter()

# Loop through each instance of mjtheme_namecode
for major_theme_nc in json_df.mjtheme_namecode:
    major_proj_theme = pd.json_normalize(major_theme_nc, max_level=1)
    
    #Loop through each dictionary in this mjtheme_namecode
    for n in major_proj_theme['name']:
        projects_by_mjtheme[n] += 1

# Print the counter 
projects_by_mjtheme


Counter({'Human development': 197,
         '': 122,
         'Economic management': 33,
         'Social protection and risk management': 158,
         'Trade and integration': 72,
         'Public sector governance': 184,
         'Environment and natural resources management': 223,
         'Social dev/gender/inclusion': 119,
         'Financial and private sector development': 130,
         'Rural development': 202,
         'Urban development': 47,
         'Rule of law': 12})

In [8]:
# Convert the Counter dictionary to a dataframe df_major_theme_count for sorting and display.
df_major_theme_count = pd.DataFrame.from_dict(projects_by_mjtheme, orient='index', columns=['Count'])
df_major_theme_count.index.name = 'Major Project Theme'

# Display the count of the 10 major project themes
df_major_theme_count.sort_values('Count', ascending=False).head(10)

,Count
Major Project Theme,
Environment and natural resources management,223
Rural development,202
Human development,197
Public sector governance,184
Social protection and risk management,158
Financial and private sector development,130
,122
Social dev/gender/inclusion,119
Trade and integration,72


## Part 3 - Handling Missing Values
As shown in Part 2, the name of the major project theme is missing in 122 instances. We can fill in the missing values based on other data.
1. Create a dictionary of major project theme codes and names.
2. Populate the missing values.
3. Find the top 10 major project themes in the completed dataset.


In [9]:
# Create a dictionary of major project theme codes and their associated names

mjtheme_dict = {}

# Loop through each instance of mjtheme_namecode
for mjtheme_nc in json_df.mjtheme_namecode:
    
    #Loop through each dictionary in this mjtheme_namecode
    for d in mjtheme_nc:
        if d['name'] != '':
            mjtheme_dict[d['code']] = d['name']
print(mjtheme_dict)

{'8': 'Human development', '1': 'Economic management', '6': 'Social protection and risk management', '5': 'Trade and integration', '2': 'Public sector governance', '11': 'Environment and natural resources management', '7': 'Social dev/gender/inclusion', '4': 'Financial and private sector development', '10': 'Rural development', '9': 'Urban development', '3': 'Rule of law'}


In [10]:
df_corrected = json_df
replacements_made = 0
i = 0
for mjtheme_nc in df_corrected.mjtheme_namecode:
    j = 0
    for d in mjtheme_nc:
        if d['name'] == '':
            df_corrected.mjtheme_namecode['name'] = mjtheme_dict[d['code']]
            replacements_made += 1
        j += 1
    i += 1
print(replacements_made)

122


C:\Users\aroge\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
print(json_df.mjtheme_namecode[0])
print(json_df.mjtheme_namecode[0][0])
print(json_df.mjtheme_namecode[0][0]['code'])
#print(json_df.mjtheme_namecode)

[{'code': '8', 'name': 'Human development'}, {'code': '11', 'name': ''}]
{'code': '8', 'name': 'Human development'}
8


As expected, 122 null values were replaced.

Now, rerun our count.

In [12]:
df.sort_values('Count', ascending=False).head(10)

NameError: name 'df' is not defined